### Use Case: "Predicting the price of wine with the Keras Functional API and Tensorflow"

Building a wide and deep network using keras (tf.keras) to predict the prices of wines from its description.

1. This Problem is well suited for wide and deep learning.
2. It involves text input and there isn't any corellation between a wines' description and it's price.

### For example:

### Input:
#### Description: 
    . powerful vanilla scents rise from the glass, but the fruit, even in this difficult vintage comes out immediately.
    . it's tart and sharp, with a strong herbal component, and the wine snaps into focus quickly with fruit, acid, tannin, herb and vanilla in equal proportion.
    . Firm and tight, still quite young, this wine needs decanting and/or further age to show its best.
    
#### Variety: Pinot Noir    


### Output:
#### Price: $45

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [ ]:
#Install tensorflow
!conda install tensorflow

In [ ]:
import itertools
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf


from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
layers = keras.layers

print("You have TensorFlow version", tf.__version__)

In [ ]:
#Data Source: https://www.kaggle.com/zynicide/wine-reviews/data

URL = "https://storage.googleapis.com/sara-cloud-ml/wine_data.csv"
path = tf.keras.utils.ger_file(URL.split('/')[-1], URL)

In [ ]:
# Convert the data into pasndas data frame
data = pd.read_csv(path)

In [ ]:
# Shuffle the data
data = data.sample(frac=1)

#Print the first 5 rows
data.head()

In [ ]:
# Do some preprocessing to limit the # of wine varieties in the dataset
data = data[pd.notnull(data['country'])]
data = data[pd.notnull(data['price'])]
data = data.drop(data.columns, axis=1)

variety_threshold = 500 #anuthing less than this will be removed
value_counts = data['variety'].value_counts()
to_remove = value_counts[value_counts <= variety_threshold].index
data.replace(to_remove, np.nan, inplace=True)
data = data[pd.notnull(data['variety'])]

In [ ]:
#Split the data into train and test
train_size = int(len(data) * .8)
print("Train size: %d" % train_size)
print("Test size: %d" % (len(data) - train_size))

In [ ]:
# Train Features
description_train = data['description'][:train_size]
variety_train = data['variety'][:train_size]

#train labels
labels_train = data['price'][:train_size]

#test features
description_test = data['description'][:train_size]
variety_test = data['variety'][:train_size]

#test labels
labels_test = data['price'][:train_size]

In [ ]:
# Create a tokenizer to preprocess our test descriptions
vocab_size = 12000 #hyperparameter experiment
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(description_train) #only fit the train


In [ ]:
# wide feature 1: sparse bag of words (bow) vocab_size_vector
description_bow_train = tokenize.texts_to_matrix(description_train)
description_bow_test = tokenize.texts_to_matrix(description_test)

In [ ]:
# wide feature 2: one-hot vector of variety categories

# Using sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(variety_train)
variety_train = encoder.transform(variety_train)
variety_test = encoder.transform(variety_test)
num_class = np.max(variety_train) + 1

# Converting labels to one-hot
variety_train = keras.utils.to_categorical(variety_train, num_classes)
variety_test = keras.utils.to_categorical(variety_test, num_classes)

In [ ]:
# Define our wide model with the functional API
bow_inputs = layers.Input(shape=(vocab_size))
variety_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs, variety_inputs])
merged_layer = layers.Dense(256, activation='relu')(merged_layer)
predictions = layers.Dense(1)(merger_layer)
wide_model = keras.Model(inputs=[bow_inputs, variety_inputs], outputs=predictions)

In [ ]:
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
print(wide_model.summary())

In [ ]:
# Deep model feature: word embeddings of wine descriptions

train_embed = tokenize.texts_to_sequences(description_train)
test_embed = tokenize.texts_to_sequences(description_test)

max_seq_length = 170
train_embed = keras.preprocessing.sequence.pad_sequences(
train_embed, maxlen = max_seq_length, padding="post")
test_embed = keras.preprocessing.sequence.pad_sequences(
test_embed, maxlen = max_seq_length, padding="post")


In [ ]:
# define our deep model with functional API

deep_inouts = layers.Input(shape=(max_seq_length,))
embedding = layers.Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)
embedding = layers.Flatten()(embedding)
embed_out = keras.Model(inputs=deep_inputs, outputs=embed_out)
deep_model = keras.Model(inouts=deep_inputs, outputs=embed_out)
print(deep_model.summary())

In [ ]:
deep_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Combining the wide and deep into one model
merged_out = layers.concatenate([wide_model.output, deep_model.output])
merger_out = layers.Dense(1)(merged_out)
combined_model = keras.Model(wide_model.input + [deep_model.input], merged_out)
print(combined_model.summary())

combine_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Run training
combined_model.fit([description_bow_train, variety_train] + [train_embed], labels_train, epochs=10, batch_size=128)

In [ ]:
combined_model.evaluate([description_bow_test, variety_test] + [test_embed], labels_test, epochs=10, batch_size=128)

In [ ]:
# generating predictions
predictions = combined_model.predict([description_bow_test, variety_test] + [test_embed])

In [ ]:
# Comparing predictions with actual values for the first few items in our test dataset
num_predictions = 40
diff = 0


for i in range(num_predictions):
    val = predictions[i]
    print(description_test.iloc[i])
    print('Predicted: ',val[0], 'Actual: ' labels_test.iloc[i], '\n')
    diff += abs(val[0] - labels_test.iloc[i])

In [ ]:
# Comparing the average difference between actual price and the model's predicted price
print('Average prediction difference: ', dif / num_predictions)